In [1]:
## Import libraries
from tensorflow import keras
import pandas as pd
from flask import Flask, request, jsonify
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib

In [2]:
## Load the model
MODEL_PATH = 'api_calls/'
rec_model = keras.models.load_model(MODEL_PATH + 'model.keras')

# Read the item_vector and the movies
item_vector = pd.read_csv(MODEL_PATH + 'item_vector.csv')
user_vector = pd.read_csv(MODEL_PATH + 'user_vector.csv')
genres_one_hot = pd.read_csv(MODEL_PATH + 'genres_one_hot.csv')
movies_df   = pd.read_csv(MODEL_PATH + 'filtered_movies.csv')

# Load the scalers
scalerUser   = joblib.load(MODEL_PATH + 'scalerUser.pkl')
scalerItem   = joblib.load(MODEL_PATH + 'scalerItem.pkl')
scalerTarget = joblib.load(MODEL_PATH + 'scalerTarget.pkl')

movies_df

,movieId,year,title,genres,imdbId,tmdbId
0,6006,2003,Just Married,Comedy|Romance,305711,12090
1,6012,2003,"Guy Thing, A",Comedy|Romance,295289,9582
2,6058,2003,Final Destination 2,Horror|Thriller,309593,9358
3,6155,2003,How to Lose a Guy in 10 Days,Comedy|Romance,251127,9919
4,6156,2003,Shanghai Knights,Action|Adventure|Comedy,300471,6038
...,...,...,...,...,...,...
1883,163056,2016,Shin Godzilla,Action|Adventure|Fantasy|Scifi,4262980,315011
1884,163599,2004,The Work and the Glory,Drama|Romance,410454,36046
1885,163601,2005,The Work and the Glory II: American Zion,Drama,457530,14631
1886,165035,2011,The Grace Card,Drama,1544600,54897


In [ ]:
app = Flask(__name__)

@app.route('/api/recommend', methods=['POST'])
def recommend():
    try:
        input_dict = request.get_json()  # Get JSON data from the request
        
        
        print('Received:')
        print(input_dict)
        keys_list_strings = list(input_dict.keys())
        keys_list_ints    = [int(item) for item in keys_list_strings]
        print(keys_list_ints)
        
        
        new_user_vector = np.zeros((1,user_vector.shape[1]-1))
        
        new_user_vector[0,0] = len(input_dict)
        new_user_vector[0,1] = sum(list(input_dict.values()))/new_user_vector[0,0]
        new_user_vector[0,2] = np.std(list(input_dict.values()), ddof=1)

        # genre_dict = 'genre_name':[genre_sum, genre_count, genre_mean] 
        genre_dict={}
        for genre in genres_one_hot.columns:
            genre_dict[genre]=[0.0, 0, 0.0]

        for i in keys_list_ints:
            row = movies_df[movies_df['movieId'] == i]
        #     print(f"Movie ID: {i},\nRow: {row}\n")
            genre_list = row['genres'].str.split('|').iloc[0]
        #     print(f"Genres List: {genre_list}")
            for item in genre_list:
                genre_dict[item][0] += input_dict[str(i)]
                genre_dict[item][1] += 1

        for val in genre_dict.values():
            if val[1] != 0:
                val[2] = val[0] / val[1]

        counter = 3
        for val in list(genre_dict.values()):
            new_user_vector[0,counter] = val[2]
            new_user_vector[0,counter+len(genres_one_hot.columns)] = val[1] / new_user_vector[0,0]
            counter += 1 

        # repeat the user vector as many times as the number of items
        new_user_vector_repeated = np.tile(new_user_vector, (len(item_vector), 1))

        # scale user vector
        new_user_vector_repeated_scaled = scalerUser.transform(new_user_vector_repeated)

        # scale item vector
        item_vector_scaled              = np.zeros((item_vector.shape[0],item_vector.shape[1]-1))
#         item_vector_scaled[:,:9]        = scalerItem.transform(item_vector.iloc[:,1:10])
#         item_vector_scaled[:,9:]        = item_vector.iloc[:,10:]
        item_vector_scaled = scalerItem.transform(item_vector.iloc[:,1:])

        # predict
        y_p = rec_model.predict([new_user_vector_repeated_scaled, item_vector_scaled])

        # unscale prediction
        y_pu = scalerTarget.inverse_transform(y_p)

        # sort the results, highest prediction first
        sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
        sorted_ypu   = y_pu[sorted_index]

        sorted_items    = movies_df.iloc[sorted_index]  #using unscaled vectors for display
        
        filtered_movies_df = sorted_items[~sorted_items['movieId'].isin(keys_list_ints)]

        recommendations = filtered_movies_df[0:10]['movieId'].tolist()

        return jsonify(recommendations)
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
